# Imports y configuraciones iniciales

In [ ]:
import warnings

from typing import List

warnings.filterwarnings("ignore")

In [ ]:
# Dejar el path principal como el anterior.
import sys

sys.path.append('../')

In [ ]:
import logging

# Crear el logger
log = logging.getLogger(__name__)

# Setear el nivel del registro
log.setLevel(logging.WARNING)

# Formato de los mensajes
formatter = logging.Formatter("%(levelname)s: (%(asctime)s) [%(filename)s: %(lineno)s] %(message)s")

if not log.hasHandlers():
    # Handlers
    file_handler = logging.FileHandler("logging.log")
    file_handler.setFormatter(formatter)  # Setear el formato del handler
    # Agregar el handler al logger
    # log.addHandler(file_handler)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    # Agregar el handler al logger
    log.addHandler(stream_handler)

Instalar la librería plot-likert y otras librerías útiles

In [ ]:
# Librería para hacer gráficos Likert
# !pip install plot-likert

# Para obtener datos de excel
# !pip install openpyxl

# Para tener un transformador de data
# !pip install -U scikit-learn

# Para tener herramientas estadísticas
# !python -m pip install statsmodels

# Para tener Seaborn
# !pip install seaborn

Empezamos importando la librería para verificar que estuvo bien instalada.

In [ ]:
import plot_likert

Importamos algunas librerías útiles para el resto del notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Importación del Data Set

In [ ]:
DATA_PATH = Path(r"..\data")

if not DATA_PATH.exists():
    log.info(f"Creando carpeta {DATA_PATH = }")
    DATA_PATH.mkdir()

In [ ]:
QNA_COMPLETE_PATH = DATA_PATH / "questions-and-answers-complete.xlsx"
QNA_COMPLETE_PATH

In [ ]:
log.debug(f"Importando datos de {QNA_COMPLETE_PATH}")
df = pd.read_excel(QNA_COMPLETE_PATH)

log.debug(f"Cantidad de respuestas: {len(df) = }")

columnas = list(df.columns)
preguntas = columnas[1:]
notas = columnas[0]

df.head()

In [ ]:
questions_ = preguntas.copy()
# Separar por subdimensiones

# Subdim 1.1
subdim_1_1 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.2
subdim_1_2 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.3
subdim_1_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0)]

# Subdim 2.1
subdim_2_1 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.2
subdim_2_2 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.3
subdim_2_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 2.4
subdim_2_4 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0), questions_.pop(0)]

del questions_

## Correlación numérica-categórica: Nota vs. Preguntas

Creando rutas y carpetas necesarias para guardar la información.

In [ ]:
# Crear la carpeta de imágenes, si no se ha creado
from pathlib import Path


IMG_PATH = Path(r"..\img")

if not IMG_PATH.exists():
    log.info(f"Creando path {IMG_PATH = }")
    IMG_PATH.mkdir()

In [ ]:
# Crear la carpeta de correlaciones, si no se ha creado
IMG_CORR_PATH = IMG_PATH / "correlations"

if not IMG_CORR_PATH.exists():
    log.info(f"Creando path {IMG_CORR_PATH = }")
    IMG_CORR_PATH.mkdir()

Buscaremos la correlación entre variables numéricas y categóricas utilizando ANOVA. Este propone como hipótesis nula lo siguiente:
\begin{equation}
H_0:\ \text{Las variables{\bf NO} están correlacionadas. (es decir, el promedio de todos los grupos{\bf son el mismo})}
\end{equation}

Este método nos arrojará un $p$-valor, el cual interpretamos de la siguiente forma: si el $p$-valor es mayor a $0.05$, entonces aceptamos la hipótesis nula $H_0$ y "concluimos" que las variables **NO** están correlacionadas.

Con las siguientes líneas de código podremos ver la correlación entre dos variables.

In [ ]:
from scipy.stats import f_oneway


def anova_corr(df, cat_name, num_name):
    # Agrupamos la variable categórica con una lista de sus respectivos valores numéricos
    category_group_lists = df.groupby(cat_name)[num_name].apply(list)
    # Aplicamos la función f_oneway para calcular los estadísticos
    anova_results = f_oneway(*category_group_lists)
    # Retornamos
    return anova_results


anova_corr(df, preguntas[0], notas)

Empezamos a calcular la matriz de correlaciones

In [ ]:
corr_cat_num = pd.DataFrame()

for i, question in enumerate(preguntas):
    corr_cat_num.loc["Notas", f"Pregunta {i+1:02d}"] = anova_corr(df, question, notas)[1]

corr_cat_num

## Gráficos

Y para graficarlos, se realizará una función que generaliza los posibles gráficos que se pueden hacer.

In [ ]:
TITULO_ESTANDAR = "Correlación ANOVA entre preguntas y nota"


def plot_cat_num_corr(
        df: pd.DataFrame, questions: List[str],
        significance: float = 0.05,
        title: str = TITULO_ESTANDAR,
        suptitle: str = None,
        add_title_significance: bool = True,
        save_path: Path = None,
        save_config: dict = None, subplots_config: dict = None,
        **kwargs
):
    log.info(f"plot_cat_num_corr({significance = }, {title = })")
    ## Setear los parámetros del subplot por default
    subplots_config = subplots_config or dict()
    subplots_config.setdefault("figsize", (1, 5))
    log.debug(f"{subplots_config = }")

    fig, ax = plt.subplots(**subplots_config)


    ## Setear parámetros del heatmap
    kwargs.setdefault("vmin", 0)
    kwargs.setdefault("vmax", significance*2)
    kwargs.setdefault("center", significance)
    kwargs.setdefault("cmap", "coolwarm")
    kwargs.setdefault("annot", True)
    kwargs.setdefault("fmt", ".1%")
    log.debug(f"{kwargs = }")

    # Setear un dataframe a plotear
    new_df: pd.DataFrame = df[questions].T

    sns.heatmap(new_df, **kwargs)


    ## Setear titulo e ylabel
    if add_title_significance:
        title += f"\nSignificancia = {significance:.1%}"

    plt.suptitle(suptitle)
    plt.title(title)
    plt.ylabel("Preguntas")


    ## Configuración para salvar imagen
    if save_path:
        log.info(f"Salvando figura en {save_path=}")
        save_config = save_config or dict()
        save_config.setdefault("dpi", 600)
        save_config.setdefault("bbox_inches", "tight")
        log.debug(f"{save_config=}")
        fig.savefig(save_path, **save_config)

    return fig, ax


plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[5:10],
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-prueba.png",
    subplots_config=dict(figsize=(1, 4)),
)

plt.show()

In [ ]:
plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[0:5],
    significance=0.1,
    title=("Sub-dimensión 1.1: Preparación de la clase"
           f"\n{TITULO_ESTANDAR}"),
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-sub-dim-1-1.png",
    subplots_config=dict(figsize=(1, 3)),
)

plt.show()

In [ ]:
plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[5:10],
    significance=0.1,
    title=("Sub-dimensión 1.2: Metodologías activas"
           f"\n{TITULO_ESTANDAR}"),
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-sub-dim-1-2.png",
    subplots_config=dict(figsize=(1, 3)),
)

plt.show()

In [ ]:
plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[10:16],
    significance=0.1,
    title=("Sub-dimensión 1.3: Evaluación de y para los aprendizajes"
           f"\n{TITULO_ESTANDAR}"),
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-sub-dim-1-3.png",
    subplots_config=dict(figsize=(1, 3.5)),
)

plt.show()

In [ ]:
plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[16:18],
    significance=0.1,
    title=("Sub-dimensión 2.1: Aprendizaje Mediado por Tecnologías"
           f"\n{TITULO_ESTANDAR}"),
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-sub-dim-2-1.png",
    subplots_config=dict(figsize=(1, 1.5)),
)

plt.show()

In [ ]:
plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[18:20],
    significance=0.1,
    title=("Sub-dimensión 2.2: Aprendizaje Directo"
           f"\n{TITULO_ESTANDAR}"),
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-sub-dim-2-2.png",
    subplots_config=dict(figsize=(1, 1.5)),
)

plt.show()

In [ ]:
plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[20:24],
    significance=0.1,
    title=("Sub-dimensión 2.3: Comunicación Pedagógica"
           f"\n{TITULO_ESTANDAR}"),
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-sub-dim-2-3.png",
    subplots_config=dict(figsize=(1, 2.5)),
)

plt.show()

In [ ]:
plot_cat_num_corr(
    corr_cat_num,
    corr_cat_num.columns[24:31],
    significance=0.1,
    title=("Sub-dimensión 2.4: Interacciones en el Trabajo Colaborativo Inclusivo"
           f"\n{TITULO_ESTANDAR}"),
    save_path=IMG_CORR_PATH / "corr-pregunta-nota-sub-dim-2-4.png",
    subplots_config=dict(figsize=(1, 3.5)),
)

plt.show()

La significancia es el grado de "libertad" en que uno puede rechazar una hipótesis nula. Si uno tiene una significancia mayor, será más permisivo en rechazar hipótesis nulas, de forma que más veces se concluirá que dos variables estarán correlacionadas. Las significancias que típicamente se utilizan son los de $10\%$, $5\%$, $2.5\%$, $1\%$ e incluso $0.1\%$

Un valor alto de $p$-valor se puede interpretar como que la pregunta no aporta información para la nota.

El valor de la escala va desde el cero hasta dos veces la significancia. Esto se hace así, porque valores más alto a dos veces la significancia, se pueden interpretar como *outliers*, y sólo afectan en la escala de colores, haciendo más difícil leerles. Es para mayor robustez.

Los colores naranjas son valores en los que **ya se aceptó** la hipótesis nula, es decir, que se concluye que las variables **no** están correlacionadas, sin embargo, se puede deber a que falta mayor información, y que siendo un poco más laxos, se puede aceptar aquella correlación como válida.

## Correlación entre variables categóricas: Preguntas vs. Preguntas

Para ver las correlaciones entre variables categóricas será necesario utilizar el test del Chi-Cuadrado.

In [ ]:
cross_tab_result = pd.crosstab(index=df[preguntas[0]], columns=df[preguntas[1]])
cross_tab_result

In [ ]:
from scipy.stats import chi2_contingency

chi_sqr_result = chi2_contingency(cross_tab_result)
chi_sqr_result

In [ ]:
def chi_sqr_corr(df, cat_name1, cat_name2):
    cross_tab_result = pd.crosstab(index=df[cat_name1], columns=df[cat_name2])
    chi_sqr_result = chi2_contingency(cross_tab_result)
    return chi_sqr_result

chi_sqr_corr(df, preguntas[0], preguntas[0])

In [ ]:
from itertools import product

chi_sqr_mat = pd.DataFrame()

for i, j in product(range(len(preguntas)), repeat=2):
    log.debug(f"Voy en la pregunta\n{i = },\n{j = }.")
    chi_sqr_mat.loc[f"P{i+1:02d}", f"P{j+1:02d}"] = chi_sqr_corr(df, preguntas[i], preguntas[j])[1]

chi_sqr_mat

## Gráficos

In [ ]:
significancia = 0.05

fig, ax = plt.subplots(figsize=(15, 15))

mask = np.triu(np.ones_like(chi_sqr_mat, dtype=np.bool))


sns.heatmap(
    chi_sqr_mat,
    vmin=0, vmax=significancia * 2,
    center=significancia,
    cmap="coolwarm",
    annot=True,
    fmt=".0%",
    mask=mask,
)

plt.title(f"Correlación Chi cuadrado entre las preguntas.\nsignificancia = {significancia:.1%}")

plt.ylabel("Preguntas")
plt.xlabel("Preguntas")

save_config = dict()
save_config.setdefault("dpi", 600)
save_config.setdefault("bbox_inches", "tight")
log.debug(f"{save_config=}")
fig.savefig(IMG_CORR_PATH / "corr-pregunta-pregunta-triang.png", **save_config)

plt.show()

In [ ]:
significancia = 0.05

fig, ax = plt.subplots(figsize=(15, 15))


sns.heatmap(
    chi_sqr_mat,
    vmin=0, vmax=significancia * 2,
    center=significancia,
    cmap="coolwarm",
    annot=True,
    fmt=".0%",
)

plt.title(f"Correlación Chi cuadrado entre las preguntas.\nsignificancia = {significancia:.1%}")

plt.ylabel("Preguntas")
plt.xlabel("Preguntas")

save_config = dict()
save_config.setdefault("dpi", 600)
save_config.setdefault("bbox_inches", "tight")
log.debug(f"{save_config=}")
fig.savefig(IMG_CORR_PATH / "corr-pregunta-pregunta-completo.png", **save_config)

plt.show()

**NO VERLO COMO VERDAD ABSOLUTA**

En un principio, interpreté que si dos preguntas estaba correlacionadas, entonces hay "información repetida", y una de las preguntas está "sobrando". Sin embargo, observando más detenidamente, me di cuenta que la pregunta 1 está correlacionada con la pregunta 2, y a su vez, la pregunta 2 está correlacionada con la pregunta 3, sin embargo, la pregunta 1 y 2 **no están correlacionadas**. Esto es algo extraño, pues si pensamos de la misma forma que al principio, uno esperaría una suerte de "transitividad".

Esto quiere decir que, las preguntas no eran tan "reemplazables" como uno esperaba al principio.

Anotar la palabra: *anticorrelación*